# Test negation detection after entity linking

This notebooks tests the results from both MedCAT and MetaCAT, as you would use it in practice.

In [1]:
from pathlib import Path
from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.config import Config
from medcat.meta_cat import MetaCAT
import configparser

/Users/stan3/Data/MedCAT/medcat/cat.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Input
data_dir = Path.cwd().parents[0] / 'data'
cdb_file = data_dir / 'cdb.dat'
vocab_file = data_dir / 'vocab.dat'

# Output
model_dir = Path.cwd().parents[0] / 'models' / 'bilstm'

# Load biLSTM from MetaCAT
meta_cat = MetaCAT.load(model_dir)

## Setup MedCAT and MetaCAT pipeline

In [3]:
# Load the cdb and vocab 
config = Config()
config.general['spacy_model'] = 'nl_core_news_lg'

vocab = Vocab.load(vocab_file)
cdb = CDB.load(cdb_file)

# Create MedCAT pipeline
cat = CAT(cdb=cdb, vocab=vocab, config=config, meta_cats=[meta_cat])

In [4]:
# Test on DL1114 from DCC with negation
text = 'Echo- en rontgenonderzoek van de heup toont geen evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9994577169418335, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9983184337615967, 'name': 'Negation'}}




In [5]:
# Test on DL1114 from DCC without negation
text = 'Echo- en rontgenonderzoek van de heup toont evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999274611473083, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999313354492188, 'name': 'Negation'}}




## Additional testing

In [6]:
# Test on DL1112 from DCC
text = 'Patient kan zich geen trauma herinneren.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Trauma is not identified as medical concept.

In [7]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/acute-buik-op-basis-van-een-wandelende-milt
text = '1 maand na de operatie had patiënte geen buikpijn meer en was zij goed hersteld.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: operatie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9998740553855896, 'name': 'Negation'}}


Entity: buikpijn
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999579787254333, 'name': 'Negation'}}




In [8]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/een-bezoar-bij-een-vrouw-met-clomipramine-intoxicatie
text = 'Er waren geen tekenen van darmobstructie te zien.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: tekenen
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999704360961914, 'name': 'Negation'}}


Entity: darmobstructie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9989364743232727, 'name': 'Negation'}}


Entity: zien
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9681671261787415, 'name': 'Negation'}}




In [9]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/nieuws/vaker-ziek-na-acute-fase-covid-19
text = 'Alle patiënten werden gematcht met een controlegroep bij wie geen SARS-CoV-2-infectie was geregistreerd.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: patiënten
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9998071193695068, 'name': 'Negation'}}


Entity: controlegroep
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9999379515647888, 'name': 'Negation'}}


Entity: SARS-CoV
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9996728301048279, 'name': 'Negation'}}


Entity: infectie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9639003276824951, 'name': 'Negation'}}




In [10]:
text = 'Er zijn geen bijwerkingen gemeld van de scan'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
# Negation was incorrectly identified
# Entity linking of bijwerkingen was missed

Entity: geen bijwerkingen
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9953070878982544, 'name': 'Negation'}}


Entity: scan
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9846878051757812, 'name': 'Negation'}}




### Check difference space at start
If this causes issues, it can be fixed using `add_prefix_space` during tokenization

In [11]:
text = " Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9468612670898438, 'name': 'Negation'}}




In [12]:
text = "Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")


Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9923927783966064, 'name': 'Negation'}}




### Some more examples

In [13]:
text = "Geen zwelling treedt op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999251365661621, 'name': 'Negation'}}




In [14]:
text = "Geen zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9998317360877991, 'name': 'Negation'}}


